# MongoDB with Python

In [11]:
import pymongo
from bson.son import SON
from pprint import pprint

In [12]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db

## 1. CRUD

### Insert

In [ ]:
# Insert one item
db.inventory.insert_one(
    {"item": "canvas",
     "qty": 100,
     "status": "C",
     "tags": ["cotton"],
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "dim_cm": [28, 35.5]})

In [ ]:
# 전부 삭제
db.inventory.delete_many({})

# Insert many items
db.inventory.insert_many([
   # MongoDB adds the _id field with an ObjectId if _id is not present
   { 
       "item": "journal", 
       "qty": 25, 
       "status": "A",
       "size": { 
           "h": 14, 
           "w": 21, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "notebook", 
       "qty": 50, 
       "status": "A",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "paper", 
       "qty": 100, 
       "status": "D",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank", "plain" ],
       "dim_cm": [8.5, 11]
   },
   { 
       "item": "planner", 
       "qty": 75, 
       "status": "D",
       "size": { 
           "h": 22.85, 
           "w": 30, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ] ,
       "dim_cm": [22.85, 30]
   },
   { 
       "item": "postcard", 
       "qty": 45, 
       "status": "A",
       "size": 
       { 
           "h": 10, 
           "w": 15.25, 
           "uom": "cm" 
       }, 
       "tags": [ "blue" ],
       "dim_cm": [10, 15.25]
   }
])

### Query

In [ ]:
# Query all documents
for item in db.inventory.find({}):
    print(item['item'])

In [ ]:
# Query exact matchs with dictionary
for item in db.inventory.find({'size' : { "h": 10, "w": 15.25, "uom": "cm" }}):
    print(item['item'], item['size'])

In [ ]:
# Query subdocument items
for item in db.inventory.find({"size.uom": "in"}):
    print(item['item'], item['size']['uom'])

In [ ]:
# SELECT * FROM inventory WHERE size.h
for item in db.inventory.find({'size.h': {'$gt' : 10 }}):
    print(item['item'], item['size']['h'])

In [ ]:
# SELECT * FROM inventory WHERE status in ("A", "D")
for item in db.inventory.find({'status': {"$in" : ['A', 'D']}}):
    print(item['item'], item['status'])

In [ ]:
# SELECT * FROM inventory WHERE status = 'D' AND qty > 50
for item in db.inventory.find({'status' : 'D', 'qty' : {'$gt' : 50 }}):
    print(item['item'], item['status'], item['qty'])

In [ ]:
# SELECT * FROM inventory WHERE status = 'C' AND qty < 50
for item in db.inventory.find({'$or' : [{'status' : 'D'}, {'qty': {'$lt' : 50}}]}):
    print(item['status'], item['qty'])

In [ ]:
# SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")
cursor = db.inventory.find({
    "status": "A",
    "$or": [{"qty": {"$lt": 30}}, {"item": {"$regex": "^p"}}]})

for item in cursor:
    print(item['status'], item['qty'], item['item'])

In [ ]:
# Exact maching
for item in db.inventory.find({"tags": ["red", "blank"]}):
    print(item['item'], item["tags"])

In [ ]:
# Query arrays which contains "red" and "black" in any order
for item in db.inventory.find({"tags": { '$all' : ["red", "blank"]}}):
    print(item['item'], item["tags"])

In [ ]:
# Query arrays which contains "red"
for item in db.inventory.find({"tags": "red"}):
    print(item['item'], item["tags"])

In [ ]:
#  Queries for all documents where the array dim_cm contains at least one element whose value is greater than 25.
for item in db.inventory.find({"dim_cm": {'$gt' : 25}}):
    print(item['item'], item["dim_cm"])

In [ ]:
# 15보다 작은 하나의 엘리먼트가 하나 이상, 20보다 큰 엘리먼트가 하나 이상 있어야 한다.
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])
print('------')    
for item in db.inventory.find({"dim_cm": {"$lt": 15, "$gt": 20}}):
    print(item['item'], item["dim_cm"])

In [ ]:
# 'Or' condition
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------')  

for item in db.inventory.find({'$or' : [{"dim_cm": {"$lt": 9}}, {"dim_cm": {"$gt": 20}}]}):
    print(item['item'], item["dim_cm"])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# dim_cm에 두 조건을 모두 만족하는 엘리먼트가 하나 이상 있어야한다.
for item in db.inventory.find({"dim_cm": {"$elemMatch": {"$gt": 22, "$lt": 30}}}):
    print(item['item'], item["dim_cm"])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# INDEX를 이용한 접근 방법
for item in db.inventory.find({'dim_cm.1': {"$gt": 25}}):
    print(item['item'], item['dim_cm'])

In [ ]:
# Array 길이
# $gt를 사용해서 2보다 큰 Array 모두 반환하는 그런 문법은 안되는 듯.
for item in db.inventory.find({'tags' : {'$size': 2}}):
    print(item['item'], item['tags'])

In [ ]:
# 1보다 큰 Array는 아래 처럼 사용하면 됨
for item in db.inventory.find({'$where' : 'this.tags.length > 1'}):
    print(item['item'], item['tags'])

In [ ]:
# 서브 document가 배열 형태로 있는 예제를 수행하기 위해 새로운 DB생성
db.inventory.delete_many({})

db.inventory.insert_many([
    {"item": "journal",
     "instock": [
         SON([("warehouse", "A"), ("qty", 5)]),
         SON([("warehouse", "C"), ("qty", 15)])]},
    {"item": "notebook",
     "instock": [
         SON([("warehouse", "C"), ("qty", 5)])]},
    {"item": "paper",
     "instock": [
         SON([("warehouse", "A"), ("qty", 60)]),
         SON([("warehouse", "B"), ("qty", 15)])]},
    {"item": "planner",
     "instock": [
         SON([("warehouse", "A"), ("qty", 40)]),
         SON([("warehouse", "B"), ("qty", 5)])]},
    {"item": "postcard",
     "instock": [
         SON([("warehouse", "B"), ("qty", 15)]),
         SON([("warehouse", "C"), ("qty", 35)])]}])

In [ ]:
# 리스트 서브 문서 중 하나의 문서라도 조건을 만족하면 반환

for item in db.inventory.find({"instock": SON([("warehouse", "A"), ("qty", 5)])}):
    print(item['item'], item['instock'])
    
print('필드 위치 바뀌면 안된다.')    

for item in db.inventory.find({"instock": SON([("qty", 5), ("warehouse", "A")])}):
    print(item['item'], item['instock'])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('- 하나의 문서라도 less than equal에 걸리면 반환')

for item in db.inventory.find({'instock.qty': {"$lte": 5}}):
    print(item['item'], item['instock'])

In [ ]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 두 조건을 모두 만족해야 함')

for item in db.inventory.find({"instock": {"$elemMatch": {"qty": {"$gt": 10, "$lte": 20}}}}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 아래 한 조건만 맞는 엘리만트가 있으면 반환')
    
for item in db.inventory.find({"instock.qty": {"$lt": 10, "$lte": 30}}):
    print(item['item'], item['instock'])
    
    

In [ ]:
for item in db.inventory.find({"instock.qty": 5, "instock.warehouse": "A"}):
    print(item['item'], item['instock'])

In [ ]:
db.inventory.delete_many({})

db.inventory.insert_many([
    {
        "item": "journal",
        "status": "A",
        "size": {"h": 14, "w": 21, "uom": "cm"},     
        "instock": [{"warehouse": "A", "qty": 5}]
    },
    {
        "item": "notebook",
        "status": "A",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "C", "qty": 5}]
    },
    {
        "item": "paper",
        "status": "D",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "A", "qty": 60}]
    }, 
    {
        "item": "planner",
        "status": "D",
        "size": {"h": 22.85, "w": 30, "uom": "cm"},
        "instock": [{"warehouse": "A", "qty": 40}]
    },
    {
        "item": "postcard",
        "status": "A",
        "size": {"h": 10, "w": 15.25, "uom": "cm"},
        "instock": [
            {"warehouse": "B", "qty": 15},
            {"warehouse": "C", "qty": 35}
        ]
    }
])


In [ ]:
# SELECT * from inventory WHERE status = "A"

for item in db.inventory.find({"status": "A"}):
    pprint(item)


In [ ]:
# SELECT _id, item, status from inventory WHERE status = "A"
#  _id, item, status 만 반환한다.

for item in db.inventory.find({"status": "A"}, {"item": 1, "status": 1}):
    pprint(item)

# _id는 제외. _id만 예외적으로 include, exclude에 섞어서 쓸 수 있다. _id를 instock으로 변경하면 에러.
for item in db.inventory.find({"status": "A"}, {"item": 1, "status": 1, '_id' : 0}):
    pprint(item)

# _id, instock, status를 제외한 나머지 반환
for item in db.inventory.find({"status": "A"}, {'_id' : 0, 'instock': 0, 'status': 0}):
    pprint(item)


In [ ]:
# .문법으로 서브 문서 접근하는 방법

for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'status': 1, 'size.uom': 1}):
    pprint(item)

In [ ]:
# .문법으로 서브 문서 배열 접근하는 방법

for item in db.inventory.find({"status": "A"}, {'_id': 0, 'instock.qty': 1}):
    pprint(item)

In [ ]:
for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'instock': 1}):
    pprint(item)
# $slice로 마지막 엘리먼트를 가지고 온다.        
for item in db.inventory.find({"status": "A"}, {'_id': 0, 'item': 1, 'instock': {'$slice': -1}}):
    pprint(item)

### Update

In [ ]:
db.inventory.delete_many({})

db.inventory.insert_many([
    {"item": "canvas",
     "qty": 100,
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "journal",
     "qty": 25,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "mat",
     "qty": 85,
     "size": {"h": 27.9, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "mousepad",
     "qty": 25,
     "size": {"h": 19, "w": 22.85, "uom": "cm"},
     "status": "P"},
    {"item": "notebook",
     "qty": 50,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "P"},
    {"item": "paper",
     "qty": 100,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "D"},
    {"item": "planner",
     "qty": 75,
     "size": {"h": 22.85, "w": 30, "uom": "cm"},
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": {"h": 10, "w": 15.25, "uom": "cm"},
     "status": "A"},
    {"item": "sketchbook",
     "qty": 80,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "sketch pad",
     "qty": 95,
     "size": {"h": 22.85, "w": 30.5, "uom": "cm"},
     "status": "A"}
])

In [ ]:
# 단일 업데이트
# {"item": "paper"} 매칭 아이템을 $set으로 수정하고 "$currentDate" 현재 시간 항목 추가


db.inventory.update_one(
    {"item": "paper"},
    {"$set": {"size.uom": "cm", "status": "P"},
     "$currentDate": {"lastModified": True}})

print (db.inventory.find_one({"item": "paper"}))

In [ ]:
# 다수 항목 수정

db.inventory.update_many(
    {"qty": {"$lt": 50}},
    {"$set": {"size.uom": "in", "status": "P"},
     "$currentDate": {"lastModified": True}})

for item in db.inventory.find({"qty": {"$lt": 50}}):
    pprint(item)

In [ ]:
# _id를 제외하고 전체 문서 교체

db.inventory.replace_one(
    {"item": "paper"},
    {"item": "paper",
     "instock": [
         {"warehouse": "A", "qty": 60},
         {"warehouse": "B", "qty": 50}]}
)

print (db.inventory.find_one({"item": "paper"}))

### Delete

In [ ]:
# 삭제

db.inventory.delete_many({"status": "A"})
db.inventory.delete_one({"status": "D"})

### Bulk Write
There are two types of bulk write operations

* Ordered : 순차 실행, 실행 중 실패하면 즉각 반환
* Unordered : 병렬 처리: 실행 중 실패해도 나머지 실행

It supports the folloings.

* insertOne
* updateOne
* updateMany
* replaceOne
* deleteOne
* deleteMany

In [ ]:
db.schools.delete_many({})

db.schools.insert_many([
    { "_id" : 1, "char" : "Brisbane", "class" : "monk", "lvl" : 4 },
    { "_id" : 2, "char" : "Eldon", "class" : "alchemist", "lvl" : 3 },
    { "_id" : 3, "char" : "Meldane", "class" : "ranger", "lvl" : 3 }
])

In [ ]:
from pymongo import InsertOne, DeleteOne, UpdateOne, ReplaceOne
from pymongo.errors import BulkWriteError

operations = [
    InsertOne({"_id" : 4, "char" : "Dithras", "class" : "barbarian", "lvl" : 4}),
    InsertOne({"_id" : 5, "char" : "Taeln", "class" : "fighter", "lvl" : 3}),
    UpdateOne({"char" : "Eldon"}, { "$set" : { "status" : "Critical Injury" } }) ,
    DeleteOne({ "char" : "Brisbane"}),
    ReplaceOne({ "char" : "Meldane" }, { "char" : "Tanys", "class" : "oracle", "lvl" : 4 })
]
    
try:    
    db.schools.bulk_write(operations, ordered=True)
except BulkWriteError as bwe:
    pprint(bwe.details)
    

### Text Search

mongodb는 문장 검색 기능을 제공한다. 문장 검색 기능을 이용하기 위해서는 index 생성이 선행되어야 한다.

In [ ]:
from pymongo import IndexModel, TEXT

db.stores.delete_many({})

for index in db.stores.index_information():
    if index == 'text_index':
        db.stores.drop_index('text_index')
    
db.stores.insert_many(
   [
     { 'name': "Java Hut", 'description': "Coffee and cakes" },
     { 'name': "Burger Buns", 'description': "Gourmet hamburgers" },
     { 'name': "Coffee Shop", 'description': "Just coffee" },
     { 'name': "Clothes Clothes Clothes", 'description': "Discount clothing" },
     { 'name': "Java Shopping", 'description': "Indonesian goods" }
   ]
)

# text 검색을 위해서는 TEXT index가 설정되어야 한다.
db.stores.create_index([('name', pymongo.TEXT), ('description', pymongo.TEXT)], name='text_index')


In [ ]:
# java, shop, discount 개별 단어를 검색하기 위한 옵션
for item in db.stores.find({'$text':{'$search':"java shop discount"}}):
    print(item['name'], '-', item['description'])
    

In [ ]:
# java, shop 개별 단어를 검색, 하지만 coffee 단어는 제외
for item in db.stores.find({'$text':{'$search':"java shop -coffee"}}):
    print(item['name'], '-', item['description'])

In [ ]:
# "java shop" 전체 단어를 검색하기 위한 옵션
for item in db.stores.find({'$text':{'$search':"\"java shop\""}}):
    print(item['name'], '-', item['description'])
    

## 2. Indexes

In [59]:
# 다음은 csv파일을 몽고 DB에 임포트 하는 방법이다.

import pandas as pd
import json

def import_movies():
    data = pd.read_csv('movies_initial.csv')
    payload = json.loads(data.to_json(orient='records'))
    db.movie_init.delete_many({})
    db.movie_init.insert_many(payload)

import_movies()

### Single field indexes

![alt text](https://docs.mongodb.com/manual/_images/index-ascending.bakedsvg.svg)

In [28]:
for index in db.movie_init.index_information():
    if index == 'title_index':
        db.movie_init.drop_index('title_index')
        
without_index = db.movie_init.find({ 'title' : 'A Turn of the Century Illusionist'}).explain()['executionStats']['executionTimeMillis']
db.movie_init.create_index([('title', pymongo.ASCENDING)], name='title_index')
with_index = db.movie_init.find({ 'title' : 'A Turn of the Century Illusionist'}).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis without index', without_index)
print('executionTimeMillis with index', with_index)

executionTimeMillis without index 27
executionTimeMillis with index 0


### Compound indexes

![alt text](https://docs.mongodb.com/manual/_images/index-compound-key.bakedsvg.svg)

In [29]:
# Delelte index
for index in db.movie_init.index_information():
    if index == 'genre_index':
        db.movie_init.drop_index('genre_index')

without_index = db.movie_init.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis without index', without_index)

# Create Single index
db.movie_init.create_index([('genre', pymongo.ASCENDING)], name='genre_index')

with_single_index = db.movie_init.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis with single index', with_single_index)

# Delete Single index
for index in db.movie_init.index_information():
    if index == 'genre_index':
        db.movie_init.drop_index('genre_index')

# Create Compound Index
db.movie_init.create_index([
    ('genre', pymongo.ASCENDING), 
    ('imdbVotes', pymongo.ASCENDING)], 
    name='genre_index')

with_compound_index = db.movie_initmovie.find(
    { 
        'genre' : 'Documentary, Short', 
        'imdbVotes': {'$gt' : 1000}
    }).explain()['executionStats']['executionTimeMillis']

print('executionTimeMillis with compound index', with_compound_index)

executionTimeMillis without index 30
executionTimeMillis with single index 1
executionTimeMillis with compound index 0


## 3. Aggregation
mongodb는 aggregation 프레임워크를 통해 데이터 파이프 라인을 제공한다.
다음 이미지는 기본적인 동작 방삭을 설명하고 있다. aggregation은 기존에 제공하도 MR의 대체한다. 주의 사항은 다음과 같다.
* 하나에 문서에 대한 16 megabytes제한이 있다. (별 문제 안될 듯)
* 파이프라인의 100megabtyes 제한이 있기 때문에 추가 메모리 필요 시, allowDiskUse 옵션을 사용해서 디스크를 사용하도록 하는 방법이 있다.

[데이터 다운로드](http://media.mongodb.org/zips.json?_ga=2.99011869.2022651543.1530572333-39471557.1528560913&_gac=1.215164773.1528560913.CjwKCAjw9e3YBRBcEiwAzjCJumSodWfu2cOhDwuxCfDXo69AJN-7fCecstC3SygLcAWoBctE5ZQrwhoCrP4QAvD_BwE)

![alt text](https://docs.mongodb.com/manual/_images/aggregation-pipeline.bakedsvg.svg)

### Projecting Queries

aggregation을 이용한 projection은 데이터 포맷을 변경하는데 강력한 기능을 제공한다.


In [35]:
# 인텍스 예제에서 사용했던 moive 데이터 사용

pipeline = [
    {
        # 상위 100개 항목만 사용
        '$limit': 100 
    },
    {
        '$project': {
            # 1은 포함 0은 불포함. _id를 제외하고 같이 사용할 수 없다.
            'title': 1,
            'year': 1,
            # string 값을 리스트 형태로 변경
            # director 키 이름을 directors로 변경
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            # fullplot이름을 fullPlot으로 변경
            'fullPlot': "$fullplot", 
            # rating이름을 rated로 변경
            'rated': "$rating",
            'released': 1,
            'runtime': 1,
            'poster': 1,
            # Embedded document로 변경
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            'lastUpdated': "$lastupdated"
        }
    },
    {
        # my_out 컬랙션이 생성된어 결과를 저장한다.
        '$out': "movies_scratch"
    }
]

db.movie_init.aggregate(pipeline)

In [42]:
db.movies_scratch.delete_many({})

pipeline = [
    {
        '$limit': 100
    },
    {
        # lastupdated 필드를 .으로 분할해서 첫번째 엘리먼트만 사용한다.
        # 두번째 엘리먼트는 다은 단계에서 인식하지 못하는 포멧이기 때문
        '$addFields': {
            'lastupdated': {
                '$arrayElemAt': [
                    {'$split': ["$lastupdated", "."]},
                    0
                ]}
        }
    },
    {
        '$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
            'released_old': '$released',
            'released': {
                # 조건문 시작
                '$cond': {
                    # $released값과 ""이 같지 않으면 then 실행
                    'if': {'$ne': ["$released", ""]},
                    'then': {
                        # $dateFromString은 날짜 문자열을 date object로 변경
                        '$dateFromString': {
                            'dateString': "$released"
                        }
                    }, # $released 값이 비어 있으면 그냥 빈값.
                    'else': ""}},
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            # 앞 단계에서 '.'뒷부분이 잘려 나간 것 확인
            'lastUpdated_old': '$lastupdated',
            # 앞 단계에서 처리된 값을 data object로 변경한다.
            'lastUpdated': {
                '$cond': {
                    'if': {'$ne': ["$lastupdated", ""]},
                    'then': {
                        '$dateFromString': {
                            'dateString': "$lastupdated",
                            'timezone': "America/New_York"
                        }
                    },
                    'else': ""}}
        }
    },
    {
        '$out': "movies_scratch"
    }
]

db.movie_init.aggregate(pipeline)

파이썬으로 다음 같이 변경하는 방법도 있다.

In [61]:
from pymongo import UpdateOne
from datetime import datetime
import re
import pprint

db.movie_init.delete_many({})
import_movies()

# '1 min' 이렇게 된 포맷에서 앞쪽 0~9숫자를 뽑아서 int변환하기 위한 코드
runtime_pat = re.compile(r'([0-9]+) min')

updates = list()

# 100개만 바꿔 봅시다.
for movie in db.movie_init.find({}).limit(100):

    fields_to_set = {}
    fields_to_unset = {}

    for k,v in movie.copy().items():
        if v == "" or v == [""] or v is None:
            del movie[k]
            fields_to_unset[k] = ""

    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
    if 'released' in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                      "%Y-%m-%d")
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")

    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset
    # pprint.pprint(update_doc)

    updates.append(UpdateOne({'_id': movie['_id']}, update_doc))

    # 다음과 같이하면 한번씩 업데이트를 해서 느리다.
    # db.movie_init.update_one({'_id': movie['_id']}, update_doc)

if len(updates) > 0:
    db.movie_init.bulk_write(updates)
    

다른 예제
string으로된 birthday를 datetime으로 변경

In [64]:
import urllib.request
from pymongo import InsertOne
from bson.json_util import loads

url = 'https://s3.amazonaws.com/edu-static.mongodb.com/lessons/coursera/building-an-app/cleansing-data-with-updates/people-raw.json'

db.people_raw.delete_many({})

batch_size = 1000
inserts = []
count = 0

with urllib.request.urlopen(url) as response:
    for line in response: 
        inserts.append(InsertOne(loads(line)))

        count += 1
                       
        if count == batch_size:
            db.people_raw.bulk_write(inserts)
            inserts = []
            count = 0
if inserts:         
    db.people_raw.bulk_write(inserts)
    count = 0

In [73]:
import dateparser

updates = []
count = 0

print(db.people_raw.find({'birthday': {'$type': 'string'}}).count());

for person in db.people_raw.find({'birthday': {'$type': 'string'}}):
    # dateparser.parse는 datetime 오브젝트를 반환한다.
    updates.append(UpdateOne({ "_id": person["_id"] }, { '$set': {"birthday": dateparser.parse(person["birthday"])}}))
    count += 1
                       
    if count == batch_size:
        db.people_raw.bulk_write(updates)
        updates = []
        count = 0
        
if updates:         
    db.people_raw.bulk_write(updates)
    count = 0
    
print(db.people_raw.find({'birthday': {'$type': 'string'}}).count());

0
